In [1]:
from deepface import DeepFace

2023-12-22 12:18:39.577056: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-22 12:18:39.657985: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-22 12:18:39.658037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-22 12:18:39.659707: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-22 12:18:39.670440: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-22 12:18:39.671501: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
import os
import cv2

image_dir = '/home/mwilkers1/Documents/Projects/Datathons/deepface-challenge/faceimages'
faceimages = []

for filename in os.listdir(image_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        faceimages.append(image)


In [7]:
import pandas as pd
from deepface import DeepFace

# Create a list to store the predictions
predictions = []

# Loop through each image in the faceimages list
for image in faceimages:
    # Perform the prediction using DeepFace
    result = DeepFace.analyze(image, actions=['gender', 'age', 'race'], enforce_detection=False)
    
    # Extract the predicted gender, age/ethnicity, and age from the result
    gender = result[0]['gender']
    age_ethnicity = result[0]['race']
    age = result[0]['age']
    
    # Append the predictions to the list
    predictions.append({'Gender': gender, 'Age/Ethnicity': age_ethnicity, 'Age': age})

# Create a dataframe from the predictions list
df_predictions = pd.DataFrame(predictions)

# Print the dataframe
print(df_predictions)


Action: race: 100%|██████████| 3/3 [00:01<00:00,  1.72it/s]  

                                                Gender  \
0    {'Woman': 71.99474573135376, 'Man': 28.0052572...   
1    {'Woman': 3.188711404800415, 'Man': 96.8112885...   
2    {'Woman': 78.93328666687012, 'Man': 21.0667163...   
3    {'Woman': 0.00011239695822951035, 'Man': 99.99...   
4    {'Woman': 0.00015127600363484817, 'Man': 99.99...   
..                                                 ...   
105  {'Woman': 0.013152977044228464, 'Man': 99.9868...   
106  {'Woman': 28.633302450180054, 'Man': 71.366697...   
107  {'Woman': 0.002035227225860581, 'Man': 99.9979...   
108  {'Woman': 1.2657110579311848, 'Man': 98.734283...   
109  {'Woman': 94.60658431053162, 'Man': 5.39341568...   

                                         Age/Ethnicity  Age  
0    {'asian': 99.96625185012817, 'indian': 0.00065...   29  
1    {'asian': 0.42595414189721, 'indian': 0.473472...   29  
2    {'asian': 82.68356323242188, 'indian': 3.50468...   31  
3    {'asian': 1.0408530591241028e-10, 'indian': 1.... 

In [9]:
# Define the filename for the CSV file
csv_filename = 'predictions.csv'

# Add the 'filename' column to the DataFrame
df_predictions['filename'] = [os.path.basename(filename) for filename in os.listdir(image_dir) if filename.endswith('.jpg') or filename.endswith('.png')]

# Replace missing predictions with 'unknown' and -1
df_predictions.fillna({'Gender': 'unknown', 'Race/Ethnicity': 'unknown', 'Age': -1}, inplace=True)

# Get the gender and race with the highest confidence for each prediction
df_predictions['Gender'] = df_predictions['Gender'].apply(lambda x: max(x, key=lambda k: x[k]))
df_predictions['Age/Ethnicity'] = df_predictions['Age/Ethnicity'].apply(lambda x: max(x, key=lambda k: x[k]))

# Reorder the columns
df_predictions = df_predictions[['filename', 'Gender', 'Age/Ethnicity', 'Age']]

# Write the DataFrame to a CSV file
df_predictions.to_csv(csv_filename, index=False)
